# Assessment with Bounding Boxes Demo

This notebook demonstrates how to use the new bounding box functionality in the IDP Assessment Service.

## Overview

The Assessment Service now supports optional bounding box extraction that can:
- Provide spatial localization of extracted fields
- Output UI-compatible geometry format
- Maintain full backward compatibility when disabled


In [ ]:
# Import required libraries
import yaml
from idp_common.assessment.service import AssessmentService
from idp_common.models import Document, Section, Page
import json

## Configuration Setup

First, let's set up the configuration with bounding boxes enabled:

In [ ]:
# Load configuration with bounding boxes enabled
with open('config/assessment_with_bounding_boxes.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded:")
print(f"Bounding boxes enabled: {config['assessment']['bounding_boxes']['enabled']}")
print(f"Model: {config['assessment']['model']}")

## Service Initialization

Initialize the Assessment Service with bounding box support:

In [ ]:
# Initialize the assessment service
assessment_service = AssessmentService(config=config)

# Check if bounding boxes are enabled
bbox_enabled = assessment_service._is_bounding_box_enabled()
print(f"Bounding box extraction is {'enabled' if bbox_enabled else 'disabled'}")

## Coordinate Conversion Demo

Demonstrate how bounding box coordinates are converted to geometry format:

In [ ]:
# Example: Convert bounding box coordinates to geometry format
example_bbox = [100, 200, 300, 400]  # [x1, y1, x2, y2] in 0-1000 scale
page_number = 1

geometry = assessment_service._convert_bbox_to_geometry(example_bbox, page_number)

print("Input coordinates:", example_bbox)
print("Converted geometry:")
print(json.dumps(geometry, indent=2))

# Show the conversion calculations
print("\nConversion details:")
print(f"left = {example_bbox[0]}/1000 = {example_bbox[0]/1000}")
print(f"top = {example_bbox[1]}/1000 = {example_bbox[1]/1000}")
print(f"width = ({example_bbox[2]}-{example_bbox[0]})/1000 = {(example_bbox[2]-example_bbox[0])/1000}")
print(f"height = ({example_bbox[3]}-{example_bbox[1]})/1000 = {(example_bbox[3]-example_bbox[1])/1000}")

## Assessment Response Processing Demo

Show how LLM responses with bounding box data are processed:

In [ ]:
# Example LLM response with bounding box data
mock_llm_response = {
    "account_number": {
        "confidence": 0.95,
        "confidence_reason": "Clear text with high OCR confidence",
        "bbox": [150, 250, 350, 300],
        "page": 1
    },
    "account_balance": {
        "confidence": 0.88,
        "confidence_reason": "Good text quality, minor formatting",
        "bbox": [400, 500, 600, 550],
        "page": 1
    },
    "customer_name": {
        "confidence": 0.92,
        "confidence_reason": "Clear name with good spacing"
        # No bbox data - will be passed through unchanged
    }
}

print("Original LLM response:")
print(json.dumps(mock_llm_response, indent=2))

# Process the response to extract geometry
processed_response = assessment_service._extract_geometry_from_assessment(mock_llm_response)

print("\nProcessed response with geometry:")
print(json.dumps(processed_response, indent=2))

## Error Handling Demo

Demonstrate how the service handles various error conditions:

In [ ]:
# Test various error conditions
error_test_cases = {
    "invalid_bbox_format": {
        "field1": {
            "confidence": 0.9,
            "confidence_reason": "Good quality",
            "bbox": "invalid_format",  # Should be list of 4 numbers
            "page": 1
        }
    },
    "missing_page": {
        "field2": {
            "confidence": 0.85,
            "confidence_reason": "Decent quality",
            "bbox": [100, 200, 300, 400]
            # Missing page number
        }
    },
    "reversed_coordinates": {
        "field3": {
            "confidence": 0.8,
            "confidence_reason": "Acceptable quality",
            "bbox": [300, 400, 100, 200],  # x2 < x1, y2 < y1 - should be corrected
            "page": 1
        }
    }
}

for test_name, test_data in error_test_cases.items():
    print(f"\n=== Testing: {test_name} ===")
    print("Input:", json.dumps(test_data, indent=2))
    
    result = assessment_service._extract_geometry_from_assessment(test_data)
    print("Result:", json.dumps(result, indent=2))

## Configuration Comparison

Compare behavior with bounding boxes enabled vs disabled:

In [ ]:
# Create two configurations: one with bounding boxes enabled, one disabled
config_enabled = {
    "assessment": {
        "bounding_boxes": {"enabled": True}
    }
}

config_disabled = {
    "assessment": {
        "bounding_boxes": {"enabled": False}
    }
}

# Test both configurations
service_enabled = AssessmentService(config=config_enabled)
service_disabled = AssessmentService(config=config_disabled)

print("Service with bounding boxes enabled:", service_enabled._is_bounding_box_enabled())
print("Service with bounding boxes disabled:", service_disabled._is_bounding_box_enabled())

# Test response processing with both services
test_response = {
    "test_field": {
        "confidence": 0.9,
        "confidence_reason": "Test data",
        "bbox": [100, 100, 200, 200],
        "page": 1
    }
}

print("\nWith bounding boxes ENABLED:")
result_enabled = service_enabled._extract_geometry_from_assessment(test_response)
has_geometry = "geometry" in result_enabled.get("test_field", {})
print(f"Geometry generated: {has_geometry}")

print("\nWith bounding boxes DISABLED:")
result_disabled = service_disabled._extract_geometry_from_assessment(test_response)
has_geometry = "geometry" in result_disabled.get("test_field", {})
print(f"Geometry generated: {has_geometry}")

## Summary

This demo showed:

1. **Configuration**: How to enable bounding box extraction
2. **Coordinate Conversion**: How bbox coordinates are converted to UI-compatible geometry format
3. **Response Processing**: How LLM responses with bounding box data are processed
4. **Error Handling**: How various error conditions are handled gracefully
5. **Backward Compatibility**: How the feature can be disabled with no impact

The bounding box integration provides powerful spatial localization capabilities while maintaining full compatibility with existing workflows.